In [ ]:
ModelName=''
BlurMethod=''
Train_acc=[0.0]
Test_acc=[0.0]
people=[0.0]
sofa=[0.0]
bed=[0.0]
chair=[0.0]
table=[0.0]
clock=[0.0]
oven=[0.0]
cat=[0.0]
dog=[0.0]
bear=[0.0]
bird=[0.0]
bicycle=[0.0]
bottle=[0.0]
keyboard=[0.0]
knife=[0.0]
epoch=0
class_list = ['people', 'sofa', 'bed', 'chair','table','clock','oven','cat','dog','bear','bird','bicycle','bottle','keyboard','knife']

In [ ]:
def classAcc(name,value) :
  if name == "people" :
    people[0]=value
  elif name=="sofa" :
    sofa[0]=value
  elif name=="bed" :
    bed[0]=value
  elif name=="chair" :
    chair[0]=value
  elif name=="table" :
    table[0]=value
  elif name=="clock" :
    clock[0]=value
  elif name=="oven" :
    oven[0]=value
  elif name=="cat" :
    cat[0]=value
  elif name=="dog" :
    dog[0]=value
  elif name=="bear" :
    bear[0]=value
  elif name=="bird" :
    bird[0]=value
  elif name=="bicycle" :
    bicycle[0]=value
  elif name=="bottle" :
    bottle[0]=value
  elif name=="keyboard" :
    keyboard[0]=value
  elif name=="knife" :
    knife[0]=value

In [ ]:
!pwd

In [ ]:
import os
path = "/content" # /content is pretty much the root. you can choose other path in your colab workspace
os.chdir(path)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import random
from keras.models import Sequential
from keras.layers import Dense
import keras

In [ ]:
#ResNet50 모델가져오기
ModelName='ResNet50'
base_model = tf.keras.applications.ResNet50(weights='imagenet', include_top=False)    # 마지막 output 출력 수정하기 위해서
print(base_model.summary())

In [ ]:
for i, layer in enumerate(base_model.layers):
  print(i, layer.name)

In [ ]:
# 클래스 수 정의
NUM_CLASSES = 15
# 끝단 layer 수정
x = base_model.output
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dense(1024, activation='relu')(x)
x = tf.keras.layers.Dropout(0.5)(x)
x = tf.keras.layers.Dense(1024, activation='relu')(x)
x = tf.keras.layers.Dense(512, activation='relu')(x)
preds = tf.keras.layers.Dense(NUM_CLASSES, activation ='softmax')(x)

model = tf.keras.models.Model(inputs=base_model.input, outputs=preds)
print(model.summary())

In [ ]:
# for i, layer in enumerate(model.layers):
#   print(i, layer.name)

for layer in model.layers[:175]:
  layer.trainable =False

for layer in model.layers[175:]:
  layer.trainable = True

In [ ]:
# verify CUDA
!/usr/local/cuda/bin/nvcc --version

In [ ]:
#preprocessing_function=tf.keras.applications.resnet50.preprocess_input,validation_split=0.2
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(preprocessing_function=tf.keras.applications.resnet50.preprocess_input,validation_split=0.2)
BlurMethod='Original' 
train_generator = train_datagen.flow_from_directory('/content/drive/MyDrive/mindoRoh_intern/mydataset/train/', 
                           target_size = (224, 224),
                           color_mode = 'rgb',
                           batch_size = 32,
                           class_mode = 'categorical',
                           shuffle=True,
                           #subset='training'
                           )
# test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale = 1./255)
# validation_generator = test_datagen.flow_from_directory('/content/drive/MyDrive/mindoRoh_intern/mydataset/test/',
#                                             target_size = (224, 224),
#                                             batch_size = 32,
#                                             class_mode = 'categorical')

validation_generator = train_datagen.flow_from_directory(
    '/content/drive/MyDrive/mindoRoh_intern/mydataset/train/', # same directory as training data
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
    ) # set as validation data

In [ ]:
#학습
#optimizer 및 lr 설정하기
#optimizer = keras.optimizers.Adam(lr=0.1) (Adam은 Momentum과 AdaGrad를 섞은 기법이므로 momentum 따로 설정 안해줘도됨!)
model.compile(optimizer= 'adadelta', loss='categorical_crossentropy', metrics=['accuracy'])

history = model.fit_generator(generator=train_generator, 
                steps_per_epoch=train_generator.n//train_generator.batch_size,
                validation_data=validation_generator,
                validation_steps = validation_generator.n // validation_generator.batch_size,
                epochs =80)

In [ ]:
Train_acc=0.9950
epoch=50
# loss 값 보고 parameter 조정하기 위해 !!!(모델 parameter 선정 근거)
acc = history.history['accuracy']
loss = history.history['loss']
val_acc = history.history['val_accuracy']
val_loss = history.history['val_loss']

plt.figure()
plt.plot(acc, label='Train Acc')
plt.plot(val_acc, label='Val Acc')
plt.ylabel('Accuracy')
plt.legend(loc='best')
plt.title('Accuracy')

plt.figure()
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Val Loss')
plt.ylabel('Loss')
plt.title('Training Loss')
plt.xlabel('epoch')
plt.legend(loc='best')
plt.xlim(0,49)
plt.show()

In [ ]:
#testset은 각 class 당 100개 (trainset은 class 당 400개)
test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(preprocessing_function=tf.keras.applications.resnet50.preprocess_input)
test_generator = test_datagen.flow_from_directory('/content/drive/MyDrive/mindoRoh_intern/mydataset/test/', 
                           target_size = (224, 224),
                           color_mode = 'rgb',
                           batch_size = 1,
                           class_mode = 'categorical',
                           shuffle = False)

pred = model.predict_generator(test_generator, verbose=1)  # make prediction

In [ ]:
predicted_class_indices = np.argmax(pred, axis=1)    # return the index of the maximum along axis
labels = (train_generator.class_indices)
labels = dict((v,k) for k,v in labels.items())
predictions = [labels[k] for k in predicted_class_indices]
filenames = test_generator.filenames
results = pd.DataFrame({"Filename":filenames, "Predictions":predictions})

len(results)

In [ ]:
c=0
nc=0
total_correct=0
test_size=100
current=0
classname=[]
value=[]
for answer, predict in zip(results['Filename'], results['Predictions']):
    current=current+1
    temp=answer.split('/')
    if temp[0]==predict :
      c=c+1
      total_correct=total_correct+1
    else :
      nc=nc+1
    if current==100 :
      #Class별 test_acc
      current=0
      classname.append(temp[0])
      value.append(c/test_size)
      print(temp[0],(c/test_size),'%')
      classAcc(temp[0],round(c/(test_size),3))
      c=0
      nc=0
#전체 test_acc
Test_acc=(total_correct/1500)
print(Test_acc)
#print(c/(c+nc))
#print(nc)

In [ ]:
plt.figure(figsize=(15, 10))
plt.bar(classname, value)
plt.title('Test Accuracy')
plt.xlabel('class')
plt.ylabel('Accuracy')
plt.ylim(0.85,1)
plt.show()

In [ ]:
#최종 결과값 엑셀파일에 저장해서 비교해보자 ! 일일히 캡처하지말자
import pandas as pd
df = pd.read_excel('result.xlsx')
df.drop(['Unnamed: 0'], axis = 1, inplace = True)
df=df.append({'Model' : ModelName ,'Blur Method':BlurMethod,'epoch' : epoch,'Train acc':Train_acc,'Test acc' : Test_acc, 'People' : people,'Sofa' : sofa,'Bed' : bed,'Chair' : chair, 'Table' : table,'Clock' : clock,'Oven' : oven,'Cat' : cat,'Dog' : dog,'Bear' : bear,'Bird' : bird , 'Bicycle' : bicycle , 'bottle' : bottle , 'keyboard' : keyboard, 'knife': knife},ignore_index=True )
df.to_excel('result.xlsx')

#####################################################----################################################

In [ ]:
BlurMethod="Original"
epoch=50

In [ ]:
import pandas as pd
#처음에만 돌리는 코드 엑셀파일 생성!
df = pd.DataFrame( columns = ['Model' , 'Blur Method','epoch', 'Train acc', 'Test acc', 'People', 'Sofa', 'Bed', 'Chair', 'Table', 'Clock', 'Oven', 'Cat', 'Dog', 'Bear', 'Bird', 'Bicycle', 'bottle', 'keyboard', 'knife'])
df.to_excel('result.xlsx')

In [ ]:
!pwd

In [ ]:
cd drive/MyDrive/mindoRoh_intern/

In [ ]:
!pwd

In [ ]:
df

In [ ]:
cd drive/MyDrive/mindoRoh_intern/